In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import deque
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from sklearn.metrics import mean_squared_error

In [ ]:
# import glucose_transformer

# from glucose_transformer import (
#     TimeSeriesDataset,
#     TransformerEncoder_version2,
#     TransformerEncoder,
#     load_ohio_series_train,
#     create_population_splits,
#     create_loocv_splits,
#     create_4fold_splits,
#     split_into_continuous_series,
#     create_train_val_datasets,
#     train_model,
#     evaluate_model,
#     evaluate_and_save_metrics_population,
#     evaluate_and_save_metrics,
#     save_model,
#     load_model
# )
%run /content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/glucose_transformer.py

# Train on DiaTrend

In [ ]:
%run /content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/glucose_transformer.py

## 5 fold CV

In [ ]:
def create_5fold_splits(data_path):
    uids = [int(file.split('.')[0].split('processed_cgm_data_Subject')[1]) for file in os.listdir(data_path)]
    # print(uids)
    splits = [0, 11, 22, 33, 44, float('inf')]
    fold_splits = {}
    for fold in range(5):
        fold_name = f"fold{fold+1}"

        # Create test and train sets
        test_files = [i for i in uids if splits[fold] < i <= splits[fold+1]]
        # print("test", test_files)
        train_files = [i for i in uids if i not in test_files]
        # print("train", train_files)
        # Add to splits dictionary
        fold_splits[fold_name] = {
            'test': ['processed_cgm_data_Subject'+str(i)+'.csv' for i in test_files],
            'train': ['processed_cgm_data_Subject'+str(i)+'.csv' for i in train_files]
        }

        # break

    return fold_splits

In [ ]:
data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/diatrend_processed/'

fold_splits = create_5fold_splits(data_dir)
print(fold_splits)

In [ ]:
def convert_to_datetime(date_str):
  try:
    return pd.to_datetime(date_str)
  except ValueError:
    return pd.to_datetime(date_str + ' 00:00:00')

In [ ]:
def load_train_data_by_fold(fold_name):
  data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/diatrend_processed/'
  train_df = pd.DataFrame()

  for file in os.listdir(data_dir):
    if file in fold_splits[fold_name]['train']:
      df = pd.read_csv(os.path.join(data_dir, file))
      uid = file.split('.')[0].split('processed_cgm_data_Subject')[1]
      df = df.rename(columns={"date": "timestamp"})
      df['USUBJID'] = [uid] * len(df)
      df['timestamp'] = df['timestamp'].apply(convert_to_datetime)

      df = df.loc[:, ['USUBJID', 'timestamp', 'mg/dl']]
      train_df = pd.concat([train_df, df])
      # break
      # print(train_df.USUBJID.unique())
  return train_df

## model train

In [ ]:
fold_lst = fold_splits.keys()
print(fold_lst)

for fold in fold_lst:
  train_df = load_train_data_by_fold(fold)
  print(fold, '\ntrain data shape:', train_df.shape)
  # break

  # Move model to GPU if available
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  # 4. Set hyperparameters
  past_sequence_length = 24
  future_offset = 6
  batch_size = 64
  max_interval_minutes = 30

  # 5. Train model
  model = TransformerEncoder_version2(
      past_seq_len=past_sequence_length,
      num_layers=1,
      d_model=512,
      nhead=4,
      input_dim=1,
      dropout=0.2
  )
  model = model.to(device)

  # Create datasets
  train_series_list = []
  for uid in train_df['USUBJID'].unique():
      cur_df = train_df[train_df['USUBJID'] == uid]
      cur_df.drop(columns=['USUBJID'], inplace=True)
      series_list = split_into_continuous_series(cur_df, past_sequence_length, future_offset, max_interval_minutes)
      train_series_list.extend(series_list)

  train_dataset, val_dataset = create_train_val_datasets(
      train_series_list,
      train_ratio=0.8,
      past_seq_len=past_sequence_length,
      future_offset=future_offset
  )

  # Create data loaders
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size)

  # Train model
  train_losses, val_losses = train_model(
      model=model,
      train_loader=train_loader,
      val_loader=val_loader,
      num_epochs=200,
      learning_rate=1e-3
  )

  model_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/'
  sh = 'sh'+str(past_sequence_length)

  # Save the trained model
  save_dir=os.path.join(model_dir, 'saved_models_diatrend/5_fold_'+sh+'/')
  os.makedirs(save_dir, exist_ok=True)
  save_model(model, sh+'_'+fold, save_dir)


## individual evaluation

In [ ]:
def evaluate_and_save_metrics_diatrend(model, test_df, save_dir="metrics",
                            past_sequence_length=7, future_offset=6,
                            batch_size=32, max_interval_minutes=30):
    """
    Evaluate model performance on test data and save metrics to file.

    Args:
        model: The trained model
        save_dir: Directory to save metrics
        past_sequence_length: Length of input sequence
        future_offset: Prediction horizon
        batch_size: Batch size for testing
        max_interval_minutes: Maximum interval between readings to consider continuous
    """
    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Split into continuous series
    test_series_list = split_into_continuous_series(test_df, past_sequence_length, future_offset,max_interval_minutes)

    # Create dataset and dataloader
    test_dataset, _ = create_train_val_datasets(
        test_series_list,
        train_ratio=0.9999,
        past_seq_len=past_sequence_length,
        future_offset=future_offset
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Evaluate model
    model.eval()
    predictions = []
    ground_truths = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to('cuda') if torch.cuda.is_available() else inputs
            targets = targets.to('cuda') if torch.cuda.is_available() else targets

            outputs = model(inputs)
            predictions.extend(outputs.cpu().numpy())
            ground_truths.extend(targets.cpu().numpy())

    # Convert to numpy arrays
    predictions = np.array(predictions).flatten()
    ground_truths = np.array(ground_truths).flatten()

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(ground_truths, predictions))
    mae = np.mean(np.abs(predictions - ground_truths))
    mape = np.mean(np.abs((ground_truths - predictions) / ground_truths)) * 100

    # Print metrics
    print(f'Test file: {uid}')
    print(f'Root Mean Square Error (RMSE): {rmse:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

    # Save metrics to file
    metrics_filename = f"metrics_{uid}.txt"
    metrics_path = os.path.join(save_dir, metrics_filename)

    with open(metrics_path, 'w') as f:
        f.write(f"Test File: {uid}\n")
        f.write(f"RMSE: {rmse:.2f}\n")
        f.write(f"MAE: {mae:.2f}\n")
        f.write(f"MAPE: {mape:.2f}%\n")

    # Create plots
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(predictions[:200], label='Predictions', color='r')
    plt.plot(ground_truths[:200], label='Ground Truth', color='b')
    plt.xlabel('Sample')
    plt.ylabel('Value')
    plt.title('Predictions vs Ground Truth')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.scatter(ground_truths, predictions, alpha=0.5)
    plt.plot([min(ground_truths), max(ground_truths)],
             [min(ground_truths), max(ground_truths)],
             'r--', label='Perfect Prediction')
    plt.xlabel('Ground Truth')
    plt.ylabel('Predictions')
    plt.title(f'Scatter Plot (RMSE: {rmse:.2f})')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return {
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'predictions': predictions,
        'ground_truths': ground_truths
    }

In [ ]:
past_sequence_length = 24
future_offset = 6
batch_size = 64
max_interval_minutes = 30
sh = 'sh'+str(past_sequence_length)
model_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/'
data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/diatrend_processed/'

test_eval = []
for fold in fold_splits.keys():
  print(fold, fold_splits[fold]['test'])
  # Load the saved model
  model = load_model_population(sh+'_'+fold, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_diatrend/5_fold_'+sh+'/'))

  for test in fold_splits[fold]['test']:
    test_df = pd.read_csv(os.path.join(data_dir, test))
    uid = test.split('.')[0].split('processed_cgm_data_Subject')[1]
    test_df = test_df.rename(columns={"date": "timestamp"})
    # test_df['USUBJID'] = [uid] * len(test_df)
    test_df['timestamp'] = test_df['timestamp'].apply(convert_to_datetime)
    test_df = test_df.loc[:, ['timestamp', 'mg/dl']]
    # print(test_df.shape)
    # break
    metrics = evaluate_and_save_metrics_diatrend(
        model=model,
        test_df=test_df,
        save_dir=os.path.join(model_dir, 'evaluation_metrics_diatrend/5_fold_individual_'+sh+'/'),
        past_sequence_length=past_sequence_length,
        future_offset=future_offset,
        batch_size=batch_size,
        max_interval_minutes=max_interval_minutes
    )

    test_eval.append([uid, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

    # print(f"\nResults for population model:")
    print(f"RMSE: {metrics['rmse']:.2f}")
    print(f"MAE: {metrics['mae']:.2f}")
    print(f"MAPE: {metrics['mape']:.2f}%")

  # break

In [ ]:
print(test_eval)
df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join(model_dir, 'evaluation_metrics_diatrend/5_fold_test_eval_'+sh+'.csv'), index=False)

# Train on T1DEXI dataset

In [ ]:
%run /content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/glucose_transformer.py

## population (archive)

### data preprocess

In [ ]:
def convert_to_datetime(date_str):
  try:
    return pd.to_datetime(date_str)
  except ValueError:
    return pd.to_datetime(date_str + ' 00:00:00')

In [ ]:
data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/T1DEXI_processed/'

train_df = pd.DataFrame()
test_df = pd.DataFrame()

for file in os.listdir(data_dir):
  if file.endswith('.csv'):
    df = pd.read_csv(os.path.join(data_dir, file))
    # df.drop(columns=['USUBJID'], inplace=True)
    df = df.rename(columns={"LBORRES": "mg/dl", "LBDTC": "timestamp"})
    df['timestamp'] = df['timestamp'].apply(convert_to_datetime)
    df = df.loc[:, ['USUBJID', 'timestamp', 'mg/dl']] # reorder to keep the same format as Diatrend for future training
    num_train = int(len(df) * 0.8)
    cur_train_df = df.iloc[:num_train]
    cur_test_df = df.iloc[num_train:]
    train_df = pd.concat([train_df, cur_train_df])
    test_df = pd.concat([test_df, cur_test_df])
    # break

population_data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/T1DEXI_population/'
train_df.to_csv(os.path.join(population_data_dir, 'T1DEXI_train.csv'), index=False)
test_df.to_csv(os.path.join(population_data_dir, 'T1DEXI_test.csv'), index=False)

In [ ]:
print(train_df.shape, test_df.shape)
train_df.head(3)

### model train

In [ ]:
%run /content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/glucose_transformer.py

In [ ]:
population_data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/T1DEXI_population/'

df = pd.read_csv(population_data_dir + 'T1DEXI_train.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 4. Set hyperparameters
past_sequence_length = 12
future_offset = 6
batch_size = 64
max_interval_minutes = 30

# 5. Train model
model = TransformerEncoder_version2(
    past_seq_len=past_sequence_length,
    num_layers=1,
    d_model=512,
    nhead=4,
    input_dim=1,
    dropout=0.2
)
model = model.to(device)

# Create datasets
train_series_list = []
for uid in df['USUBJID'].unique():
    cur_df = df[df['USUBJID'] == uid]
    cur_df.drop(columns=['USUBJID'], inplace=True)
    series_list = split_into_continuous_series(cur_df, past_sequence_length, future_offset, max_interval_minutes)
    train_series_list.extend(series_list)

train_dataset, val_dataset = create_train_val_datasets(
    train_series_list,
    train_ratio=0.8,
    past_seq_len=past_sequence_length,
    future_offset=future_offset
)

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Train model
train_losses, val_losses = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=200,
    learning_rate=1e-3
)

In [ ]:
model_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/'
sh = 'sh'+str(past_sequence_length)

# Save the trained model
save_model(model, 'population_'+sh, save_dir=os.path.join(model_dir, 'saved_models_T1DEXI/'))

### individaul evaluation

In [ ]:
def evaluate_and_save_metrics_T1DEXI(model, test_df, save_dir="metrics",
                            past_sequence_length=7, future_offset=6,
                            batch_size=32, max_interval_minutes=30):
    """
    Evaluate model performance on test data and save metrics to file.

    Args:
        model: The trained model
        save_dir: Directory to save metrics
        past_sequence_length: Length of input sequence
        future_offset: Prediction horizon
        batch_size: Batch size for testing
        max_interval_minutes: Maximum interval between readings to consider continuous
    """
    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Split into continuous series
    test_series_list = split_into_continuous_series(test_df, past_sequence_length, future_offset,max_interval_minutes)

    # Create dataset and dataloader
    test_dataset, _ = create_train_val_datasets(
        test_series_list,
        train_ratio=0.99,
        past_seq_len=past_sequence_length,
        future_offset=future_offset
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Evaluate model
    model.eval()
    predictions = []
    ground_truths = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to('cuda') if torch.cuda.is_available() else inputs
            targets = targets.to('cuda') if torch.cuda.is_available() else targets

            outputs = model(inputs)
            predictions.extend(outputs.cpu().numpy())
            ground_truths.extend(targets.cpu().numpy())

    # Convert to numpy arrays
    predictions = np.array(predictions).flatten()
    ground_truths = np.array(ground_truths).flatten()

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(ground_truths, predictions))
    mae = np.mean(np.abs(predictions - ground_truths))
    mape = np.mean(np.abs((ground_truths - predictions) / ground_truths)) * 100

    # Print metrics
    print(f'Test file: {uid}')
    print(f'Root Mean Square Error (RMSE): {rmse:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

    # Save metrics to file
    metrics_filename = f"metrics_{uid}.txt"
    metrics_path = os.path.join(save_dir, metrics_filename)

    with open(metrics_path, 'w') as f:
        f.write(f"Test File: {uid}\n")
        f.write(f"RMSE: {rmse:.2f}\n")
        f.write(f"MAE: {mae:.2f}\n")
        f.write(f"MAPE: {mape:.2f}%\n")

    # Create plots
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(predictions[:200], label='Predictions', color='r')
    plt.plot(ground_truths[:200], label='Ground Truth', color='b')
    plt.xlabel('Sample')
    plt.ylabel('Value')
    plt.title('Predictions vs Ground Truth')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.scatter(ground_truths, predictions, alpha=0.5)
    plt.plot([min(ground_truths), max(ground_truths)],
             [min(ground_truths), max(ground_truths)],
             'r--', label='Perfect Prediction')
    plt.xlabel('Ground Truth')
    plt.ylabel('Predictions')
    plt.title(f'Scatter Plot (RMSE: {rmse:.2f})')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return {
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'predictions': predictions,
        'ground_truths': ground_truths
    }

In [ ]:
past_sequence_length = 12
future_offset = 6
batch_size = 64
max_interval_minutes = 30
sh = 'sh'+str(past_sequence_length)

# Load the saved model
model = load_model_population('population_'+sh, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_T1DEXI'))
test_eval = []

test_df = pd.read_csv(population_data_dir + 'T1DEXI_test.csv')
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])

# for test in population_splits['test']:
#     print(test)
for uid in test_df['USUBJID'].unique():
    cur_df = test_df[test_df['USUBJID'] == uid]
    cur_df.drop(columns=['USUBJID'], inplace=True)
    # Evaluate on test data individually
    metrics = evaluate_and_save_metrics_T1DEXI(
        model=model,
        test_df=cur_df,
        save_dir=os.path.join(model_dir, 'evaluation_metrics_T1DEXI/80_20_individual_'+sh+'/'),
        past_sequence_length=past_sequence_length,
        future_offset=future_offset,
        batch_size=batch_size,
        max_interval_minutes=max_interval_minutes
    )

    id = uid
    test_eval.append([id, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

    # print(f"\nResults for population model:")
    print(f"RMSE: {metrics['rmse']:.2f}")
    print(f"MAE: {metrics['mae']:.2f}")
    print(f"MAPE: {metrics['mape']:.2f}%")

In [ ]:
print(test_eval)
df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join(model_dir, 'evaluation_metrics_T1DEXI/80_20_test_eval_'+sh+'.csv'), index=False)

## 5-fold cross validation

### data split

In [ ]:
# def create_5fold_splits(data_path):
#     uids = [file for file in os.listdir(data_path)]
#     total_files = len(uids)

#     # Calculate files per fold (rounded up for the first folds)
#     files_per_fold = total_files // 5
#     remainder = total_files % 5

#     # Create splits
#     fold_splits = {}
#     start_idx = 0

#     for fold in range(5):
#         fold_name = f"fold{fold+1}"

#         # Calculate number of files for this fold's test set
#         if fold < remainder:
#             current_fold_size = files_per_fold + 1
#         else:
#             current_fold_size = files_per_fold

#         # Get test indices for this fold
#         end_idx = start_idx + current_fold_size
#         test_indices = list(range(start_idx, end_idx))
#         # print(test_indices)

#         # Create test and train sets
#         test_files = [uids[i] for i in test_indices]
#         train_files = [path for i, path in enumerate(uids) if i not in test_indices]

#         # Add to splits dictionary
#         fold_splits[fold_name] = {
#             'test': test_files,
#             'train': train_files
#         }

#         # Update start index for next fold
#         start_idx = end_idx
#         # break

#     return fold_splits

In [ ]:
def create_5fold_splits(data_path):
    uids = [int(file.split('.')[0]) for file in os.listdir(data_path)]
    print(uids)
    splits = [0, 248, 1201, 1348, 1459, float('inf')]
    fold_splits = {}
    for fold in range(5):
        fold_name = f"fold{fold+1}"

        # Create test and train sets
        test_files = [i for i in uids if splits[fold] < i <= splits[fold+1]]
        # print(test_files)
        train_files = [i for i in uids if i not in test_files]

        # Add to splits dictionary
        fold_splits[fold_name] = {
            'test': [str(i)+'.csv' for i in test_files],
            'train': [str(i)+'.csv' for i in train_files]
        }

        # break

    return fold_splits

In [ ]:
data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/T1DEXI_processed/'

fold_splits = create_5fold_splits(data_dir)
print(fold_splits)

In [ ]:
def convert_to_datetime(date_str):
  try:
    return pd.to_datetime(date_str)
  except ValueError:
    return pd.to_datetime(date_str + ' 00:00:00')

In [ ]:
def load_train_data_by_fold(fold_name):
  data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/T1DEXI_processed/'
  train_df = pd.DataFrame()

  for file in os.listdir(data_dir):
    if file in fold_splits[fold_name]['train']:
      df = pd.read_csv(os.path.join(data_dir, file))
      # df.drop(columns=['USUBJID'], inplace=True)
      df = df.rename(columns={"LBORRES": "mg/dl", "LBDTC": "timestamp"})
      df['timestamp'] = df['timestamp'].apply(convert_to_datetime)
      df = df.loc[:, ['USUBJID', 'timestamp', 'mg/dl']] # reorder to keep the same format as Diatrend for future training

      train_df = pd.concat([train_df, df])
      # break
  return train_df

### model train

In [ ]:
%run /content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/glucose_transformer.py

In [ ]:
fold_lst = fold_splits.keys()
print(fold_lst)

for fold in fold_lst:
  train_df = load_train_data_by_fold(fold)
  print(fold, '\ntrain data shape:', train_df.shape)

  # Move model to GPU if available
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  # 4. Set hyperparameters
  past_sequence_length = 24
  future_offset = 6
  batch_size = 64
  max_interval_minutes = 30

  # 5. Train model
  model = TransformerEncoder_version2(
      past_seq_len=past_sequence_length,
      num_layers=1,
      d_model=512,
      nhead=4,
      input_dim=1,
      dropout=0.2
  )
  model = model.to(device)

  # Create datasets
  train_series_list = []
  for uid in train_df['USUBJID'].unique():
      cur_df = train_df[train_df['USUBJID'] == uid]
      cur_df.drop(columns=['USUBJID'], inplace=True)
      series_list = split_into_continuous_series(cur_df, past_sequence_length, future_offset, max_interval_minutes)
      train_series_list.extend(series_list)

  train_dataset, val_dataset = create_train_val_datasets(
      train_series_list,
      train_ratio=0.8,
      past_seq_len=past_sequence_length,
      future_offset=future_offset
  )

  # Create data loaders
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size)

  # Train model
  train_losses, val_losses = train_model(
      model=model,
      train_loader=train_loader,
      val_loader=val_loader,
      num_epochs=200,
      learning_rate=1e-3
  )

  model_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/'
  sh = 'sh'+str(past_sequence_length)

  # Save the trained model
  save_dir=os.path.join(model_dir, 'saved_models_T1DEXI/5_fold_'+sh+'/')
  os.makedirs(save_dir, exist_ok=True)
  save_model(model, sh+'_'+fold, save_dir)

  # break

### individual evaluation

In [ ]:
def evaluate_and_save_metrics_T1DEXI(model, test_df, save_dir="metrics",
                            past_sequence_length=7, future_offset=6,
                            batch_size=32, max_interval_minutes=30):
    """
    Evaluate model performance on test data and save metrics to file.

    Args:
        model: The trained model
        save_dir: Directory to save metrics
        past_sequence_length: Length of input sequence
        future_offset: Prediction horizon
        batch_size: Batch size for testing
        max_interval_minutes: Maximum interval between readings to consider continuous
    """
    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Split into continuous series
    test_series_list = split_into_continuous_series(test_df, past_sequence_length, future_offset,max_interval_minutes)

    # Create dataset and dataloader
    test_dataset, _ = create_train_val_datasets(
        test_series_list,
        train_ratio=0.9999,
        past_seq_len=past_sequence_length,
        future_offset=future_offset
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Evaluate model
    model.eval()
    predictions = []
    ground_truths = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to('cuda') if torch.cuda.is_available() else inputs
            targets = targets.to('cuda') if torch.cuda.is_available() else targets

            outputs = model(inputs)
            predictions.extend(outputs.cpu().numpy())
            ground_truths.extend(targets.cpu().numpy())

    # Convert to numpy arrays
    predictions = np.array(predictions).flatten()
    ground_truths = np.array(ground_truths).flatten()

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(ground_truths, predictions))
    mae = np.mean(np.abs(predictions - ground_truths))
    mape = np.mean(np.abs((ground_truths - predictions) / ground_truths)) * 100

    # Print metrics
    print(f'Test file: {uid}')
    print(f'Root Mean Square Error (RMSE): {rmse:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

    # Save metrics to file
    metrics_filename = f"metrics_{uid}.txt"
    metrics_path = os.path.join(save_dir, metrics_filename)

    with open(metrics_path, 'w') as f:
        f.write(f"Test File: {uid}\n")
        f.write(f"RMSE: {rmse:.2f}\n")
        f.write(f"MAE: {mae:.2f}\n")
        f.write(f"MAPE: {mape:.2f}%\n")

    # Create plots
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(predictions[:200], label='Predictions', color='r')
    plt.plot(ground_truths[:200], label='Ground Truth', color='b')
    plt.xlabel('Sample')
    plt.ylabel('Value')
    plt.title('Predictions vs Ground Truth')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.scatter(ground_truths, predictions, alpha=0.5)
    plt.plot([min(ground_truths), max(ground_truths)],
             [min(ground_truths), max(ground_truths)],
             'r--', label='Perfect Prediction')
    plt.xlabel('Ground Truth')
    plt.ylabel('Predictions')
    plt.title(f'Scatter Plot (RMSE: {rmse:.2f})')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return {
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'predictions': predictions,
        'ground_truths': ground_truths
    }

In [ ]:
# def load_test_data_individual(fold_name):
#   data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/T1DEXI_processed/'
#   train_df = pd.DataFrame()

#   for file in os.listdir(data_dir):
#     if file in fold_splits[fold_name]['train']:
#       df = pd.read_csv(os.path.join(data_dir, file))
#       # df.drop(columns=['USUBJID'], inplace=True)
#       df = df.rename(columns={"LBORRES": "mg/dl", "LBDTC": "timestamp"})
#       df['timestamp'] = df['timestamp'].apply(convert_to_datetime)
#       df = df.loc[:, ['USUBJID', 'timestamp', 'mg/dl']] # reorder to keep the same format as Diatrend for future training

#       train_df = pd.concat([train_df, df])
#       # break
#   return train_df

In [ ]:
past_sequence_length = 24
future_offset = 6
batch_size = 64
max_interval_minutes = 30
sh = 'sh'+str(past_sequence_length)
model_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/'
data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/T1DEXI_processed/'

test_eval = []
for fold in fold_splits.keys():
  print(fold, fold_splits[fold]['test'])
  # Load the saved model
  model = load_model_population(sh+'_'+fold, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_T1DEXI/5_fold_'+sh+'/'))

  for test in fold_splits[fold]['test']:
    uid = test.split('.')[0]
    test_df = pd.read_csv(os.path.join(data_dir, test))
    test_df = test_df.rename(columns={"LBORRES": "mg/dl", "LBDTC": "timestamp"})
    test_df['timestamp'] = test_df['timestamp'].apply(convert_to_datetime)
    test_df = test_df.loc[:, ['timestamp', 'mg/dl']]
    # print(test_df.shape)
    # break
    metrics = evaluate_and_save_metrics_T1DEXI(
        model=model,
        test_df=test_df,
        save_dir=os.path.join(model_dir, 'evaluation_metrics_T1DEXI/5_fold_individual_'+sh+'/'),
        past_sequence_length=past_sequence_length,
        future_offset=future_offset,
        batch_size=batch_size,
        max_interval_minutes=max_interval_minutes
    )

    test_eval.append([uid, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

    # print(f"\nResults for population model:")
    print(f"RMSE: {metrics['rmse']:.2f}")
    print(f"MAE: {metrics['mae']:.2f}")
    print(f"MAPE: {metrics['mape']:.2f}%")

  # break

In [ ]:
print(test_eval)
df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join(model_dir, 'evaluation_metrics_T1DEXI/5_fold_test_eval_'+sh+'.csv'), index=False)

# Train on Ohio dataset

In [ ]:
%run /content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/glucose_transformer.py

## Population data

In [ ]:
data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/'
folder_path_train_2018 = os.path.join(data_dir, "./OhioT1DM 2020/2018/train")
folder_path_train_2020 = os.path.join(data_dir,"./OhioT1DM 2020/2020/train")
train_files_2018 = [f for f in os.listdir(folder_path_train_2018) if f.endswith('.xml')]
train_files_2020 = [f for f in os.listdir(folder_path_train_2020) if f.endswith('.xml')]

folder_path_test_2018 = os.path.join(data_dir,"./OhioT1DM 2020/2018/test")
folder_path_test_2020 = os.path.join(data_dir,"./OhioT1DM 2020/2020/test")
test_files_2018 = [f for f in os.listdir(folder_path_test_2018) if f.endswith('.xml')]
test_files_2020 = [f for f in os.listdir(folder_path_test_2020) if f.endswith('.xml')]

population_splits = create_population_splits(
    folder_path_train_2018,
    folder_path_train_2020,
    train_files_2018,
    train_files_2020,
    folder_path_test_2018,
    folder_path_test_2020,
    test_files_2018,
    test_files_2020
)

print(population_splits)

In [ ]:
# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 4. Set hyperparameters
past_sequence_length = 24
future_offset = 6
batch_size = 64
max_interval_minutes = 30

# 5. Train model
model = TransformerEncoder_version2(
    past_seq_len=past_sequence_length,
    num_layers=1,
    d_model=512,
    nhead=4,
    input_dim=1,
    dropout=0.2
)
model = model.to(device)

# Load and process training data
train_dfs = []
for train_file in population_splits['train']:
    df = load_ohio_series_train(train_file, "glucose_level", "value")
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    train_dfs.append(df)

# Create datasets
train_series_list = []
for df in train_dfs:
    series_list = split_into_continuous_series(df, past_sequence_length, future_offset, max_interval_minutes)
    train_series_list.extend(series_list)

train_dataset, val_dataset = create_train_val_datasets(
    train_series_list,
    train_ratio=0.8,
    past_seq_len=past_sequence_length,
    future_offset=future_offset
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Train model
train_losses, val_losses = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=200,
    learning_rate=1e-3
)


In [ ]:
model_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/'
sh = 'sh'+str(past_sequence_length)

# Save the trained model
save_model(model, 'population_'+sh, save_dir=os.path.join(model_dir, 'saved_models_original_ohio/'))

In [ ]:
past_sequence_length = 24
future_offset = 6
batch_size = 64
max_interval_minutes = 30

sh = 'sh'+str(past_sequence_length)
model = load_model_population('population_'+sh, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_original_ohio'))

# Evaluate on test data
metrics = evaluate_and_save_metrics_population(
    model=model,
    test_file_path=population_splits['test'],
    save_dir=os.path.join(model_dir, 'evaluation_metrics_original_ohio'),
    past_sequence_length=past_sequence_length,
    future_offset=future_offset,
    batch_size=batch_size,
    max_interval_minutes=max_interval_minutes
)

# evaluation on whole test set
print(f"RMSE: {metrics['rmse']:.2f}")
print(f"MAE: {metrics['mae']:.2f}")
print(f"MAPE: {metrics['mape']:.2f}%")

In [ ]:
# Load the saved model
model = load_model_population('population_'+sh, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_original_ohio'))
test_eval = []

for test in population_splits['test']:
    print(test)
    # Evaluate on test data individually
    metrics = evaluate_and_save_metrics(
        model=model,
        test_file_path=test,
        save_dir=os.path.join(model_dir, 'evaluation_metrics_original_ohio/individual_'+sh+'/'),
        past_sequence_length=past_sequence_length,
        future_offset=future_offset,
        batch_size=batch_size,
        max_interval_minutes=max_interval_minutes
    )

    id = test.split('/')[-1].split('-')[0]
    test_eval.append([id, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

    # print(f"\nResults for population model:")
    print(f"RMSE: {metrics['rmse']:.2f}")
    print(f"MAE: {metrics['mae']:.2f}")
    print(f"MAPE: {metrics['mape']:.2f}%")

### save individual results to .csv file

In [ ]:
print(test_eval)
df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join(model_dir, 'evaluation_metrics_original_ohio/individual_test_eval_'+sh+'.csv'), index=False)

# Ohio 60 mins model train on diatrend and T1DEXI

In [ ]:
def convert_to_datetime(date_str):
  try:
    return pd.to_datetime(date_str)
  except ValueError:
    return pd.to_datetime(date_str + ' 00:00:00')

In [ ]:
past_sequence_length = 12
future_offset = 6
batch_size = 64
max_interval_minutes = 30

model_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/'

sh = 'sh'+str(past_sequence_length)
model = load_model_population('population_'+sh, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_original_ohio'))

## Diatrend

In [ ]:
def evaluate_and_save_metrics_diatrend(model, test_df, save_dir="metrics",
                            past_sequence_length=7, future_offset=6,
                            batch_size=32, max_interval_minutes=30):
    """
    Evaluate model performance on test data and save metrics to file.

    Args:
        model: The trained model
        save_dir: Directory to save metrics
        past_sequence_length: Length of input sequence
        future_offset: Prediction horizon
        batch_size: Batch size for testing
        max_interval_minutes: Maximum interval between readings to consider continuous
    """
    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Split into continuous series
    test_series_list = split_into_continuous_series(test_df, past_sequence_length, future_offset,max_interval_minutes)

    # Create dataset and dataloader
    test_dataset, _ = create_train_val_datasets(
        test_series_list,
        train_ratio=0.9999,
        past_seq_len=past_sequence_length,
        future_offset=future_offset
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Evaluate model
    model.eval()
    predictions = []
    ground_truths = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to('cuda') if torch.cuda.is_available() else inputs
            targets = targets.to('cuda') if torch.cuda.is_available() else targets

            outputs = model(inputs)
            predictions.extend(outputs.cpu().numpy())
            ground_truths.extend(targets.cpu().numpy())

    # Convert to numpy arrays
    predictions = np.array(predictions).flatten()
    ground_truths = np.array(ground_truths).flatten()

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(ground_truths, predictions))
    mae = np.mean(np.abs(predictions - ground_truths))
    mape = np.mean(np.abs((ground_truths - predictions) / ground_truths)) * 100

    # Print metrics
    print(f'Test file: {uid}')
    print(f'Root Mean Square Error (RMSE): {rmse:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

    # Save metrics to file
    metrics_filename = f"metrics_{uid}.txt"
    metrics_path = os.path.join(save_dir, metrics_filename)

    with open(metrics_path, 'w') as f:
        f.write(f"Test File: {uid}\n")
        f.write(f"RMSE: {rmse:.2f}\n")
        f.write(f"MAE: {mae:.2f}\n")
        f.write(f"MAPE: {mape:.2f}%\n")

    # Create plots
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(predictions[:200], label='Predictions', color='r')
    plt.plot(ground_truths[:200], label='Ground Truth', color='b')
    plt.xlabel('Sample')
    plt.ylabel('Value')
    plt.title('Predictions vs Ground Truth')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.scatter(ground_truths, predictions, alpha=0.5)
    plt.plot([min(ground_truths), max(ground_truths)],
             [min(ground_truths), max(ground_truths)],
             'r--', label='Perfect Prediction')
    plt.xlabel('Ground Truth')
    plt.ylabel('Predictions')
    plt.title(f'Scatter Plot (RMSE: {rmse:.2f})')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return {
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'predictions': predictions,
        'ground_truths': ground_truths
    }

In [ ]:
past_sequence_length = 12
future_offset = 6
batch_size = 64
max_interval_minutes = 30

model_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_ML4H/GlucoseTransformer/gdrive_version/'

sh = 'sh'+str(past_sequence_length)
model = load_model_population('population_'+sh, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_original_ohio'))

In [ ]:
data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/diatrend_processed/'
test_eval = []

for test in os.listdir(data_dir):
  test_df = pd.read_csv(os.path.join(data_dir, test))
  uid = test.split('.')[0].split('processed_cgm_data_Subject')[1]
  test_df = test_df.rename(columns={"date": "timestamp"})
  # test_df['USUBJID'] = [uid] * len(test_df)
  test_df['timestamp'] = test_df['timestamp'].apply(convert_to_datetime)
  test_df = test_df.loc[:, ['timestamp', 'mg/dl']]
  # print(test_df.shape)
  # break
  metrics = evaluate_and_save_metrics_diatrend(
      model=model,
      test_df=test_df,
      save_dir=os.path.join(model_dir, 'evaluation_metrics_original_ohio/individual_t1dexi_diatrend/diatrend_'+sh+'/'),
      past_sequence_length=past_sequence_length,
      future_offset=future_offset,
      batch_size=batch_size,
      max_interval_minutes=max_interval_minutes
  )

  test_eval.append([uid, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

  # print(f"\nResults for population model:")
  print(f"RMSE: {metrics['rmse']:.2f}")
  print(f"MAE: {metrics['mae']:.2f}")
  print(f"MAPE: {metrics['mape']:.2f}%")


In [ ]:
print(test_eval)
df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join(model_dir, 'evaluation_metrics_original_ohio/individual_t1dexi_diatrend/diatrend_eval_'+sh+'.csv'), index=False)

## T1DEXI

In [ ]:
def evaluate_and_save_metrics_T1DEXI(model, test_df, save_dir="metrics",
                            past_sequence_length=7, future_offset=6,
                            batch_size=32, max_interval_minutes=30):
    """
    Evaluate model performance on test data and save metrics to file.

    Args:
        model: The trained model
        save_dir: Directory to save metrics
        past_sequence_length: Length of input sequence
        future_offset: Prediction horizon
        batch_size: Batch size for testing
        max_interval_minutes: Maximum interval between readings to consider continuous
    """
    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Split into continuous series
    test_series_list = split_into_continuous_series(test_df, past_sequence_length, future_offset,max_interval_minutes)

    # Create dataset and dataloader
    test_dataset, _ = create_train_val_datasets(
        test_series_list,
        train_ratio=0.9999,
        past_seq_len=past_sequence_length,
        future_offset=future_offset
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Evaluate model
    model.eval()
    predictions = []
    ground_truths = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to('cuda') if torch.cuda.is_available() else inputs
            targets = targets.to('cuda') if torch.cuda.is_available() else targets

            outputs = model(inputs)
            predictions.extend(outputs.cpu().numpy())
            ground_truths.extend(targets.cpu().numpy())

    # Convert to numpy arrays
    predictions = np.array(predictions).flatten()
    ground_truths = np.array(ground_truths).flatten()

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(ground_truths, predictions))
    mae = np.mean(np.abs(predictions - ground_truths))
    mape = np.mean(np.abs((ground_truths - predictions) / ground_truths)) * 100

    # Print metrics
    print(f'Test file: {uid}')
    print(f'Root Mean Square Error (RMSE): {rmse:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

    # Save metrics to file
    metrics_filename = f"metrics_{uid}.txt"
    metrics_path = os.path.join(save_dir, metrics_filename)

    with open(metrics_path, 'w') as f:
        f.write(f"Test File: {uid}\n")
        f.write(f"RMSE: {rmse:.2f}\n")
        f.write(f"MAE: {mae:.2f}\n")
        f.write(f"MAPE: {mape:.2f}%\n")

    # Create plots
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(predictions[:200], label='Predictions', color='r')
    plt.plot(ground_truths[:200], label='Ground Truth', color='b')
    plt.xlabel('Sample')
    plt.ylabel('Value')
    plt.title('Predictions vs Ground Truth')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.scatter(ground_truths, predictions, alpha=0.5)
    plt.plot([min(ground_truths), max(ground_truths)],
             [min(ground_truths), max(ground_truths)],
             'r--', label='Perfect Prediction')
    plt.xlabel('Ground Truth')
    plt.ylabel('Predictions')
    plt.title(f'Scatter Plot (RMSE: {rmse:.2f})')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return {
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'predictions': predictions,
        'ground_truths': ground_truths
    }

In [ ]:
test_eval = []
# for fold in fold_splits.keys():
#   print(fold, fold_splits[fold]['test'])
data_dir = '/content/drive/Shareddrives/Yanjun/ReproGenBG/ReproGenBG_Dataset/T1DEXI_processed/'

for test in os.listdir(data_dir):
  # print(test)
  # # for test in fold_splits[fold]['test']:
  uid = test.split('.')[0]
  test_df = pd.read_csv(os.path.join(data_dir, test))
  test_df = test_df.rename(columns={"LBORRES": "mg/dl", "LBDTC": "timestamp"})
  test_df['timestamp'] = test_df['timestamp'].apply(convert_to_datetime)
  test_df = test_df.loc[:, ['timestamp', 'mg/dl']]
  # print(test_df.shape)
  # break
  metrics = evaluate_and_save_metrics_T1DEXI(
      model=model,
      test_df=test_df,
      save_dir=os.path.join(model_dir, 'evaluation_metrics_original_ohio/individual_t1dexi_diatrend/t1dexi_'+sh+'/'),
      past_sequence_length=past_sequence_length,
      future_offset=future_offset,
      batch_size=batch_size,
      max_interval_minutes=max_interval_minutes
  )

  test_eval.append([uid, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

  # print(f"\nResults for population model:")
  print(f"RMSE: {metrics['rmse']:.2f}")
  print(f"MAE: {metrics['mae']:.2f}")
  print(f"MAPE: {metrics['mape']:.2f}%")

In [ ]:
print(test_eval)
df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join(model_dir, 'evaluation_metrics_original_ohio/individual_t1dexi_diatrend/t1dexi_eval_'+sh+'.csv'), index=False)